In [ ]:
!pip install -q /kaggle/input/iterative-stratification/iterative-stratification-master

In [ ]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pylab import rcParams
import seaborn as sns

In [ ]:
cls = ["Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance"]

train = pd.read_csv("../input/siimcovid19-512-jpg-image-dataset/train.csv")
train["file_path"] = "../input/siimcovid19-512-jpg-image-dataset/train/" + train["image_id"] + ".jpg"
train["class_name"] = np.argmax(train[cls].values, axis=1)

display(train.head())

In [ ]:
train['class_name'].hist()

In [ ]:
class CFG:
    
    apex=True
    debug=False
    train=True
    print_freq=50
    target_size=len(cls)
    target_col=cls
    trn_fold=[0, 1, 2, 3]
    batch_size=64
    n_fold=4
    
    model_name='tf_efficientnet_b5'
    
    #weights=[None]*n_fold
    
    weights=["../input/siimcovidpytorchdataset/models/tf_efficientnet_b5_ns_224_lr3e5_bs32_10epo_best_fold2.pth",
             "../input/siimcovidpytorchdataset/models/tf_efficientnet_b5_ns_224_lr3e5_bs32_10epo_best_fold2.pth",
             "../input/siimcovidpytorchdataset/models/tf_efficientnet_b5_ns_224_lr3e5_bs32_10epo_best_fold2.pth",
             "../input/siimcovidpytorchdataset/models/tf_efficientnet_b5_ns_224_lr3e5_bs32_10epo_best_fold2.pth"]
    
    size=224
    
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=4
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=6 # CosineAnnealingLR
    #T_0=6 # CosineAnnealingWarmRestarts
    lr=1e-3
    min_lr=1e-5
    weight_decay=1e-5
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    num_workers=4
    freeze=False
    
if CFG.debug:
    
    CFG.epochs = 1
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Python        : ' + sys.version.split('\n')[0])
print('Numpy         : ' + np.__version__)
print('Pandas        : ' + pd.__version__)
print('PyTorch       : ' + torch.__version__)
print('Albumentations: ' + A.__version__)
print('Timm          : ' + timm.__version__)

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = average_precision_score(y_true, y_pred)
    return score * (2/3)


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [ ]:
y = train[cls].values
X = train['image_id'].values

train['fold'] = np.nan

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, random_state=CFG.seed, shuffle=True)
for i, (_, test_index) in enumerate(mskf.split(X, y)):
    train.iloc[test_index, -1] = i
    
train['fold'] = train['fold'].astype('int')
display(train.groupby(['fold', 'class_name']).size())

In [ ]:
class TrainDataset(Dataset):
    
    def __init__(self, df, transforms=None):
        
        super().__init__()
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
            
        row = self.df.iloc[idx]
        img = cv2.imread(row.file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        else:
            img = torch.from_numpy(img).float()
             
        label = row[CFG.target_col].astype('int').values
        label = torch.as_tensor(label)   
        return img, label.float()

In [ ]:
def get_transforms():    
    
    transforms_train = A.Compose([
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        #A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, border_mode=0, p=0.5),
        #A.Resize(int(CFG.size*1.25), int(CFG.size*1.25)),
        A.Resize(CFG.size,CFG.size),
        A.Cutout(max_h_size=int(CFG.size * 0.12), max_w_size=int(CFG.size * 0.12), num_holes=3, p=0.5),
        A.Normalize(),
        ToTensorV2()
    ])
    
    transforms_val = A.Compose([
        A.Resize(CFG.size, CFG.size),
        A.Normalize(),
        ToTensorV2()
    ])
    
    return transforms_train, transforms_val

In [ ]:
transforms_train, transforms_valid = get_transforms()
transforms_train

In [ ]:
ds = TrainDataset(df=train, transforms=transforms_train)
ds[0][1]

In [ ]:
rcParams['figure.figsize'] = 15,5
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = i*5 + p
        img, label = ds[idx]
        axarr[p].imshow(img.transpose(0,1).transpose(1,2).squeeze())


In [ ]:
class CustomModel(nn.Module):
    
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.base = timm.create_model(self.cfg.model_name, pretrained=pretrained)
        self.dropout = nn.Dropout(0.5)
        
        if "res" in self.cfg.model_name:
            self.myfc = nn.Linear(self.base.fc.in_features, self.cfg.target_size)
            self.base.global_pool = nn.Identity()
            self.base.fc = nn.Identity()
            
        elif "vit" in self.cfg.model_name:
            self.myfc = nn.Linear(self.base.head.in_features, self.cfg.target_size)
            self.base.global_pool = nn.Identity()
            self.base.head = nn.Identity()
            
        elif "nfnet" in self.cfg.model_name:
            self.myfc = nn.Linear(self.base.head.fc.in_features, self.cfg.target_size)
            self.base.global_pool = nn.Identity()
            self.base.head.fc = nn.Identity()
            
        elif "ception" in self.cfg.model_name:
            self.myfc = nn.Linear(self.base.last_linear.in_features, self.cfg.target_size)
            self.base.global_pool = nn.Identity()
            self.base.last_linear = nn.Identity()
        
        elif "efficient" in self.cfg.model_name:
            self.myfc = nn.Linear(self.base.classifier.in_features, self.cfg.target_size)
            self.base.classifier = nn.Identity()
            
        else:
            self.myfc = nn.Linear(self.base.classifier.in_features, self.cfg.target_size)
            self.base.global_pool = nn.Identity()
            self.base.classifier = nn.Identity()
          
        if self.cfg.freeze:
            for param in self.base.parameters():
                param.requires_grad = False
        else:
            for param in self.base.parameters():
                param.requires_grad = True
                
    def extract(self, x):
        return self.base(x)

    def forward(self, x):
        
        if "efficient" in self.cfg.model_name:
            x = self.extract(x)
            h = self.myfc(self.dropout(x))
            return h
        else:
            bs = x.size(0)
            features = self.base(x)
            pooled_features = self.pooling(features).view(bs, -1)
            output = self.myfc(pooled_features)
            return output
    

In [ ]:
m = CustomModel(cfg=CFG, pretrained=True)
x = torch.stack([ds[i][0] for i in range(2)])
y = m(x)
y.shape

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    if CFG.apex:
        scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        if CFG.apex:
            with autocast():
                y_preds = model(images)
                loss = criterion(y_preds, labels)
        else:
            y_preds = model(images)
            loss = criterion(y_preds, labels)
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            if CFG.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [ ]:
# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_col].values
    
    transform_train, transform_valid = get_transforms()

    train_dataset = TrainDataset(train_folds, 
                                 transforms=transform_train)
    valid_dataset = TrainDataset(valid_folds, 
                                 transforms=transform_valid)

    train_loader = DataLoader(train_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, 
                                          verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, 
                                                    last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, pretrained=True)
    if CFG.weights[fold] is not None:
        print(f"Loading pre-trained weights for fold-{fold}.......")
        model.load_state_dict(torch.load(CFG.weights[fold], map_location=device))
    model.to(device)

    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.BCEWithLogitsLoss()

    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_score.pth')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth')
    
    valid_folds[["Pred_"+s for s in CFG.target_col]] = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth', 
                                      map_location=torch.device('cpu'))['preds']

    return valid_folds

In [ ]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df[["Pred_"+s for s in CFG.target_col]].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)

In [ ]:
if __name__ == '__main__':
    main()